In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

In [54]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 100

Step 1: Read in data<br>
using TF Learn's built in function to load MNIST data to the folder data/mnist

In [3]:
mnist = input_data.read_data_sets('./data/mnist', one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


Step 2: create placeholders for features and labels<br>
each image in the MNIST data is of shape 28*28 = 784<br>
therefore, each image is represented with a 1x784 tensor<br>
there are 10 classes for each image, corresponding to digits 0 - 9.<br>
Features are of the type float, and labels are of the type int<br>

In [4]:
X = tf.placeholder(tf.float64)
y = tf.placeholder(tf.float64)

Step 3: create weights and bias<br>
weights and biases are initialized to 0<br>
shape of w depends on the dimension of X and Y so that Y = X * w + b<br>
shape of b depends on Y<br>

In [6]:
mnist.train.images.shape

(55000, 784)

In [7]:
mnist.train.labels.shape

(55000, 10)

In [20]:
W = tf.Variable(name="W", dtype=tf.float64, initial_value=np.zeros((784,10)))
b = tf.Variable(name="b", dtype=tf.float64, initial_value=np.zeros((1,10)))

Step 4: build model<br>
the model that returns the logits.<br>
this logits will be later passed through softmax layer<br>
to get the probability distribution of possible label of the image<br>
DO NOT DO SOFTMAX HERE<br>

In [47]:
y_pred = tf.add(tf.matmul(X, W), b)

Step 5: define loss function<br>
use cross entropy loss of the real labels with the softmax of logits<br>
use the method:<br>
tf.nn.softmax_cross_entropy_with_logits(logits, Y)<br>
then use tf.reduce_mean to get the mean loss of the batch<br>

In [48]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y))

Step 6: define training op<br>
using gradient descent to minimize loss

In [49]:
train_op = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [55]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, loss_batch = sess.run([train_op, loss], feed_dict={X: X_batch, y: Y_batch})
            total_loss += loss_batch
        
        if i % 10 == 0 and i != 0:
            print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        
    print('Total time: {0} seconds'.format(time.time() - start_time))
    print('Optimization Finished!') # should be around 0.35 after 25 epochs

    # test the model
    preds = tf.nn.softmax(y_pred)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
    
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, y:Y_batch}) 
        total_correct_preds += accuracy_batch[0]

print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 10: 0.401461039244
Average loss epoch 20: 0.356695533175
Average loss epoch 30: 0.338146802787
Average loss epoch 40: 0.320578171226
Average loss epoch 50: 0.313765778997
Average loss epoch 60: 0.305718949317
Average loss epoch 70: 0.300461717477
Average loss epoch 80: 0.297689540025
Average loss epoch 90: 0.288626796202
Total time: 35.2967209816 seconds
Optimization Finished!
Accuracy 0.918
